In [64]:
import time
import jax.numpy as jnp
import numpy as np
import torch
from jax import jit

In [2]:
## Enormous Attention Calc Comparison

In [61]:
def test(func, **kwrgs):
    cons = []
    for _ in range(100):
        start = time.time()
        func(**kwrgs)
        end = time.time()
        cons.append(end-start)

    return sum(cons ) / len(cons)

In [62]:
k = np.random.randn(256, 8000)
v = np.random.randn(256, 8000)
q = np.random.randn(256, 8000)

def softmax(x:np.array):
    x = x-x.max(axis=-1, keepdims=True)
    exp_x = np.exp(x)
    return exp_x/np.sum(exp_x, axis=-1, keepdims=True)

def np_attention(k, q, v):
    softmax(np.matmul(k,v.T)/np.sqrt(q.shape[-1])) @ v

test(np_attention, k=k, q=q, v=v)

0.06883242607116699

In [63]:
torch_k = torch.from_numpy(k)
torch_v = torch.from_numpy(v)
torch_q = torch.from_numpy(q)

def attention(k, v, q):
    with torch.no_grad():
        return torch.softmax(torch_k@torch_v.T/np.sqrt(q.shape[-1]), dim=-1) @ v

test(attention, k=torch_k, v=torch_v, q=torch_q)

0.07076051712036133

In [78]:
from jax import random

key = random.PRNGKey()

k = random.uniform(key, [256, 8000])
v = random.uniform(key, [256, 8000])
q = random.uniform(key, [256, 8000])

@jit
def softmax(x:jnp.array):
    x = x-x.max(axis=-1, keepdims=True)
    exp_x = jnp.exp(x)
    return exp_x/jnp.sum(exp_x, axis=-1, keepdims=True)

@jit
def np_attention(k, q, v):
    kq = jnp.dot(q,k.T)
    return jnp.dot(softmax(kq), v.T)

test(np_attention, k=k, q=q, v=v)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/yoonseonghyeon/opt/anaconda3/envs/mlenv/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3398, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/pk/4stgl7rx1rd40xnz4plktxy00000gn/T/ipykernel_54275/1058087367.py", line 3, in <cell line: 3>
    key = random.PRNGKey(0)
  File "/Users/yoonseonghyeon/opt/anaconda3/envs/mlenv/lib/python3.10/site-packages/jax/_src/random.py", line 136, in PRNGKey
    raise TypeError(
  File "/Users/yoonseonghyeon/opt/anaconda3/envs/mlenv/lib/python3.10/site-packages/jax/_src/prng.py", line 269, in seed_with_impl
  File "/Users/yoonseonghyeon/opt/anaconda3/envs/mlenv/lib/python3.10/site-packages/jax/_src/prng.py", line 578, in random_seed
  File "/Users/yoonseonghyeon/opt/anaconda3/envs/mlenv/lib/python3.10/site-packages/jax/_src/numpy/lax_numpy.py", line 2070, in asarray
    out_array: Array = lax_internal._convert_element_type(out, dtype, weak_type=weak